In [20]:
import pandas as pd
import numpy as np

# EXCEL - DF CSV CONVERSION
# trainData = pd.read_excel('dataset/cases_2021_train_processed_2.xlsx', sheet_name=None)
trainfile = pd.read_excel ('dataset/cases_2021_train_processed_2.xlsx') 
testfile = pd.read_excel ('dataset/cases_2021_test_processed_unlabelled_2.xlsx') 
# Write the dataframe object into csv file 
trainfile.to_csv ('dataset/Train.csv',  
                  index = None, 
                  header=True)
testfile.to_csv ('dataset/Test.csv',  
                  index = None, 
                  header=True) 

In [22]:
import pandas as pd
import numpy as np

# FEATURE MAPPING
# read csv file and convert into a dataframe object 
trainData = pd.DataFrame(pd.read_csv('dataset/Train.csv')) 
testData = pd.DataFrame(pd.read_csv('dataset/Test.csv')) 
# map function
def mapFeatures(df):
    # Map 'sex' to binary values
    sex_mapping = {"male": 0, "female": 1}
    df["sex"] = df["sex"].map(sex_mapping)

    # Map 'province' to numbers from 0 
    province_mapping = {province: idx for idx, province in enumerate(df['province'].unique())}
    df['province'] = df['province'].map(province_mapping)
    
    # Map chronic disease
    cdb_mapping = {False: 0, True: 1}
    df['chronic_disease_binary'] = df['chronic_disease_binary'].map(cdb_mapping)
    
    # map countries
    country_mapping = {country: idx for idx, country in enumerate(df['country'].unique())}
    df['country'] = df['country'].map(country_mapping)
    
mapFeatures(trainData)
mapFeatures(testData)
# Map outcome group for train data
og_mapping = {"deceased": 0,"hospitalized": 1,"nonhospitalized":2}
trainData['outcome_group'] = trainData['outcome_group'].map(og_mapping)
#print(trainData.head(10))
print(testData.head(10))

   age  sex  province  country  latitude  longitude date_confirmation  \
0   59    1         0        0  14.59580  120.97720        2020-03-31   
1   79    0         1        1  11.13927   79.08428        2020-05-24   
2   44    1         1        1  13.08362   80.28252        2020-05-19   
3   36    0         1        1  13.12462   79.91815        2020-04-30   
4   52    0         2        1  25.31258   86.48888        2020-04-24   
5   28    1         1        1  12.68224   79.98008        2020-05-31   
6   17    0         2        1  25.73271   86.98845        2020-05-23   
7   18    0         3        1  13.00287   76.10245        2020-05-24   
8   68    0         0        0  14.63000  121.03000        2020-04-11   
9   21    0         1        1  13.08362   80.28252        2020-05-15   

   chronic_disease_binary  Confirmed  Deaths  Recovered  Active  \
0                       0     747288   13297     603746  130245   
1                       0     886673   12719     858075   1587